## Load Data From DB

In [1]:
import sqlalchemy
from data import settings
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

engine = sqlalchemy.create_engine(settings.DB_TYPE + settings.DB_USER + ":" + settings.DB_PASSWORD + "@" + settings.DB_URL + ":" + settings.DB_PORT + "/" + settings.DB_NAME, echo=settings.QUERY_ECHO)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

class Reviews(Base):
    # table name:
    __tablename__ = 'reviews'
    # table column:
    idx = Column(Integer, primary_key=True)
    app_id = Column(String)
    app_name = Column(String)
    review_id = Column(String)
    title = Column(String)
    author = Column(String)
    author_url = Column(String)
    version = Column(String)
    rating = Column(String)
    review = Column(String)
    category = Column(String)

Base.metadata.create_all(engine)

In [2]:
cols = ['app_id', 'app_name', 'title', 'review', 'rating', 'category', 'author_url', 'author']
data_dict = {i:[] for i in cols}
for row in session.query(Reviews.app_id, Reviews.app_name, Reviews.title, Reviews.review, Reviews.rating, 
                         Reviews.category, Reviews.author_url, Reviews.author):
    for i, col_name in enumerate(cols):
        data_dict[col_name].append(row[i])

In [3]:
data = pd.DataFrame(data_dict)
del data_dict

In [4]:
data.shape

(616758, 8)

In [5]:
data.category.unique()

array(['엔터테인먼트', '교육', '음악', '의학', '라이프 스타일', '게임', '뉴스', '날씨', '여행',
       '사진 및 비디오', '카탈로그', '소셜 네트워킹', '생산성', '건강 및 피트니스', '비즈니스', '스포츠',
       '내비게이션', '유틸리티', '도서', '참고', '금융', 'NaN', '음식 및 음료', '잡지 및 신문',
       'None'], dtype=object)

In [6]:
print('total reviews for each category\n-------------------------------')
for cat in data.category.unique():
    print('{0}: {1}'.format(cat, data.loc[data.category == cat].shape[0]))

total reviews for each category
-------------------------------
엔터테인먼트: 35808
교육: 33328
음악: 31417
의학: 5231
라이프 스타일: 45196
게임: 59374
뉴스: 10721
날씨: 12504
여행: 28900
사진 및 비디오: 45749
카탈로그: 2052
소셜 네트워킹: 39442
생산성: 47275
건강 및 피트니스: 39093
비즈니스: 23781
스포츠: 5246
내비게이션: 17922
유틸리티: 35598
도서: 24916
참고: 16883
금융: 41898
NaN: 594
음식 및 음료: 13309
잡지 및 신문: 509
None: 12


In [7]:
from collections import Counter, defaultdict
from konlpy.tag import Twitter, Kkma, Mecab
from pprint import pprint
import ujson
twitter = Twitter()
kkma = Kkma()
mecab = Mecab()
word_counter = Counter()

새로 생긴 명사를 추출한 후 트위터 분석기로 다시 분석?

In [8]:
data.columns

Index(['app_id', 'app_name', 'author', 'author_url', 'category', 'rating',
       'review', 'title'],
      dtype='object')

**문제정의**

리뷰로 앱의 평점을예측?

**가정:**

1) 평점과 리뷰는 정의 상관관계

## 잔데이터 처리

In [5]:
docs = data.loc[:, ['app_id', 'review', 'rating']]
#docs = data.review.values.flatten()
del data

In [50]:
mask = docs.iloc[:, 0].str.len() > 10

In [55]:
docs.loc[docs.iloc[:, 0].str.len() > 10, :]

,app_id,review,rating
5960,좀 더 고급 단어들도 추가되면 좋겠습니다.,NaN,NaN
5961,그리고 검색 화면에서는 가능한 링크 기능이 플래시 카드상에서는 안되는 부분도 개선되...,NaN,NaN
8194,"하지만 와이파이 아주 잘 잡히는 환경에서만 사용 가능할듯. 너무느림ㅠㅠ,.. 최대의...",NaN,NaN
8195,아직 반도 못봤다.. 언제쯤 바로크 넘어가려나:),NaN,NaN
12569,"쉽고 재미있어요. 무료버전 다운받아 사용하다가 재미있어서 유료도 했는데,",NaN,NaN
12570,"사진도 많고 당분간 지루하진 않을듯, 앞으로 더 많은 사집 업뎃 부탁합니다.",NaN,NaN
17455,아래분 처럼 2개 열심히 넣고 나니 프로그램이 튕겨버립니다.,NaN,NaN
17456,"1달러.... 돈안까운건 그렇다치고, 정보입력하느라 삽질한 걸 생각하니 분하네요.",NaN,NaN
17457,버전업 되서 버그픽스 되기전까지는 절대로 구입하지 마세요.,NaN,NaN
17462,입력 다해놓고 잘못 됐다고 나오네요. 어서 수정을..,NaN,NaN


In [66]:
mask = ~(docs.iloc[:, 1] == 'NaN')

In [72]:
docs = docs.loc[mask, :]

In [79]:
mask2 = ~(docs.iloc[:, 1] == 'None')

In [80]:
docs = docs.loc[mask2, :]

In [84]:
docs.loc[docs.iloc[:, 0].str.len() > 10, :]

,app_id,review,rating


## train, test split

In [87]:
id_list = docs.loc[:, 'app_id'].unique()

In [ ]:
id_list = docs.loc[:, 'app_id'].unique()
train_idx = int(len(id_list) * 0.8)
train_app_id = id_list[train_idx]
train_data_idx = docs.app_id[docs.app_id.isin([train_app_id])].index[-1]
train_data = docs.iloc[:train_data_idx, :]
test_data = docs.iloc[train_data_idx:, :]
len(train_data), len(test_data)

change to tsv

In [97]:
train_data.to_csv('./data/train_docs.txt', sep='\t', index=False)
test_data.to_csv('./data/test_docs.txt', sep='\t', index=False)

---

save_to_json 은 py 파일 참고